## 导入所需的包

In [1]:
import functools

import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
train_file_path = "./datas/titanic/train.csv"
test_file_path = "./datas/titanic/test.csv"

In [3]:
# 让 numpy 数据更易读。
np.set_printoptions(precision=3, suppress=True)

## 加载数据

In [4]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [5]:
# 标签列
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [6]:
def get_dataset(file_path):
    """
    构建tensorflow的数据集格式
    """
    dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
    return dataset

# 将train和test的csv，分别加载成tensorflow的对象的格式
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [7]:
# 测试一个批次
examples, labels = next(iter(raw_train_data))
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'male', b'female', b'male', b'male', b'female', b'male',
       b'female', b'female', b'male', b'male', b'male', b'female'],
      dtype=object)>), ('age', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([34.,  5., 29., 48., 52., 25., 28., 21., 25., 28., 28.,  2.],
      dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([1, 2, 1, 0, 1, 0, 0, 0, 1, 2, 0, 0], dtype=int32)>), ('parch', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)>), ('fare', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([26.   , 19.258,  7.046,  7.854, 78.267,  7.65 ,  7.55 ,  7.75 ,
       91.079, 21.679, 14.5  , 12.288], dtype=float32)>), ('class', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'Second', b'Third', b'Third', b'Third', b'First', b'Third',
       b'Third', b'Third', b'First', b'Third', b'Third', b'Third'],


## 数据预处理

### 分类数据

In [8]:
# 分类数据的码表
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [9]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    # 提供码表的特征输入
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [10]:
# 分类特征列
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

### 连续数据

In [11]:
def process_continuous_data(mean, data):
    # 标准化数据的函数
    data = tf.cast(data, tf.float32) * 1/(2*mean)
    return tf.reshape(data, [-1, 1])

In [12]:
# 提前算好的均值
MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

numerical_columns = []

for feature in MEANS.keys():
    num_col = tf.feature_column.numeric_column(
        feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
    numerical_columns.append(num_col)

In [13]:
# 连续特征列的列表
numerical_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x10cbcbe60>, 29.631308)),
 NumericColumn(key='n_siblings_spouses', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x10cbcbe60>, 0.545455)),
 NumericColumn(key='parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x10cbcbe60>, 0.379585)),
 NumericColumn(key='fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x10cbcbe60>, 34.385399))]

### 创建预处理层

将这两个特征列的集合相加，并且传给 `tf.keras.layers.DenseFeatures` 从而创建一个进行预处理的输入层。

In [14]:
preprocessing_layer = tf.keras.layers.DenseFeatures(
    categorical_columns+numerical_columns)

## 构建模型

从 `preprocessing_layer` 开始构建 `tf.keras.Sequential`。

In [15]:
# 构建一个DNN模型
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])



In [17]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

## 训练、评估和预测

现在可以实例化和训练模型。

In [18]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

In [19]:
model.fit(train_data, epochs=20)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Epoch 1/20
53/53 [==============================] - 1s 25ms/step - loss: 0.5447 - accuracy: 0.7289
Epoch 2/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4249 - accuracy: 0.8166
Epoch 3/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4126 - accuracy: 0.8182
Epoch 4/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4039 - accuracy: 0.8246
Epoch 5/20
53/53 [==============================] - 0s 3ms/step - loss: 0.3853 - accuracy: 0.8325
Epoch 6/20
53/53 [==============================] - 0s 3ms/step - loss: 0.3909 - accuracy: 0.8293
Epoch 7/20
53/53 [==============================] - 0s 4ms/step - loss: 0.3731 - accuracy: 0.8405
Epoch 8/20
53/53 [==============================] - 0s 4ms/step -

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features (DenseFeature multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  3200      
_________________________________________________________________
dense_1 (Dense)              multiple                  16512     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  65        
Total params: 28,033
Trainable params: 28,033
Non-trainable params: 0
_________________________________________________________________


当模型训练完成的时候，你可以在测试集 `test_data` 上检查准确性。

In [21]:
test_loss, test_accuracy = model.evaluate(test_data)

print()
print(f'Test Loss {test_loss}, Test Accuracy {test_accuracy}')

     22/Unknown - 0s 20ms/step - loss: 0.4777 - accuracy: 0.7992
Test Loss 0.4777101115746932, Test Accuracy 0.7992424368858337


使用 `tf.keras.Model.predict` 推断一个批次或多个批次的标签。

In [24]:
predictions = model.predict(test_data)
predictions[:10]

array([[0.52 ],
       [0.088],
       [0.055],
       [0.966],
       [0.094],
       [0.197],
       [0.1  ],
       [0.021],
       [0.541],
       [0.548]], dtype=float32)

In [25]:
# 显示部分结果
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Predicted survival: 52.05%  | Actual outcome:  SURVIVED
Predicted survival: 8.81%  | Actual outcome:  SURVIVED
Predicted survival: 5.51%  | Actual outcome:  DIED
Predicted survival: 96.55%  | Actual outcome:  SURVIVED
Predicted survival: 9.36%  | Actual outcome:  SURVIVED
Predicted survival: 19.71%  | Actual outcome:  DIED
Predicted survival: 9.97%  | Actual outcome:  SURVIVED
Predicted survival: 2.13%  | Actual outcome:  DIED
Predicted survival: 54.12%  | Actual outcome:  DIED
Predicted survival: 54.81%  | Actual outcome:  SURVIVED
